# NIN块

In [1]:
import time
import torch
from torch import nn, optim
import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU()
    )

    return blk

In [2]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(0.5),
    nin_block(384, 10, kernel_size=3, stride=1, padding=1),
    d2l.GlobalAvgPool2d(),
    d2l.FlattenLayer()
)

X = torch.rand(1, 1, 224, 224)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape : ', X.shape)

0 output shape :  torch.Size([1, 96, 54, 54])
1 output shape :  torch.Size([1, 96, 26, 26])
2 output shape :  torch.Size([1, 256, 26, 26])
3 output shape :  torch.Size([1, 256, 12, 12])
4 output shape :  torch.Size([1, 384, 12, 12])
5 output shape :  torch.Size([1, 384, 5, 5])
6 output shape :  torch.Size([1, 384, 5, 5])
7 output shape :  torch.Size([1, 10, 5, 5])
8 output shape :  torch.Size([1, 10, 1, 1])
9 output shape :  torch.Size([1, 10])


In [3]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

lr, num_epochs = 0.002, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.7571, train acc 0.353, test acc 0.694, time 138.2 sec
epoch 2, loss 0.7112, train acc 0.742, test acc 0.778, time 137.4 sec
epoch 3, loss 0.5249, train acc 0.804, test acc 0.815, time 137.1 sec
epoch 4, loss 0.4569, train acc 0.830, test acc 0.838, time 137.4 sec
epoch 5, loss 0.4173, train acc 0.844, test acc 0.847, time 138.8 sec
